In [2]:
from numpy import *
from random  import *
from copy  import *

#Cardinal directions            
cardinal = {
    'F' : array([ 1, 0, 0], dtype=int32),
    'U' : array([ 0, 1, 0], dtype=int32),
    'R' : array([ 0, 0, 1], dtype=int32),
    'B' : array([-1, 0, 0], dtype=int32),
    'D' : array([ 0,-1, 0], dtype=int32),
    'L' : array([ 0, 0,-1], dtype=int32),
}

def getname(a):
    n=0
    for i in range(3):
        if a[i] == 0:
            n+=1
        else:
            assert(abs(a[i]) == 1)
    assert(n==2)
    
    if a[0] > 0:
        return "+X"
    if a[0] < 0:
        return "-X"
    if a[1] > 0:
        return "+Y"
    if a[1] < 0:
        return "-Y"
    if a[2] > 0:
        return "+Z"
    if a[2] < 0:
        return "-Z"
    
#Make spaces using cross product
def lookat(forward_up_name):
    f = cardinal[forward_up_name[0]]
    u = cardinal[forward_up_name[1]]
    c = cross(f,u)
    M = array([f,u,c],dtype=int32).transpose()
    N=""

    N+=getname(f)
    N+=getname(u)
    N+=getname(c)

    return N, M
    
#combinations - Take every cardinal direction combine with four 90 degree rotations
cmb = []
#forward
cmb += ["FU", "FR", "FD", "FL"]
#backward
cmb += ["BU", "BL", "BD", "BR"]
#Right
cmb += ["RU", "RB", "RD", "RF"]
#Left
cmb += ["LU", "LF", "LD", "LB"]
#Up
cmb += ["UB", "UR", "UF", "UL"]
#Down
cmb += ["DF", "DR", "DB", "DL"]

assert(len(list(set(cmb))) == 24)

O = [lookat(x) for x in cmb]

base = array([cardinal['F'],cardinal['U'],cardinal['R'], array([1,2,3], dtype=int32)])

spaces = []
for n,o in O:
    spaces.append(n)
    if 0:
        print(n)
        print(o)
        print("Base")
        b = base.dot(o)
        print("X:", b[0])
        print("Y:", b[1])
        print("Z:", b[2])
        print("Diag:", b[3])
        print("\n")
print("Spaces:", " ".join(spaces))    
        
#match two arrays of points
#return offset from A2B used if 12 matches found, else return None
def _match(A,B):    
    for a in A:
        for b in B:        
            # compute offset from a to b
            a2b_offset = b-a
            
            #count matches using offset
            m=0
            for aa in A + a2b_offset:
                if (aa==B).all(axis=1).any():
                    m+=1
                    if m > 1:
                        #print("Reached match: ", m, c)
                        pass                    
                    if m >= 12:
                        return a2b_offset
            assert(m>0)
    return None


def match(A,B):    
    for a in A:
        for b in B:        
            # compute offset from a to b
            a2b_offset = b-a            
            C = A + a2b_offset            
            nrows, ncols = C.shape
            dtype={'names':['f{}'.format(i) for i in range(ncols)],'formats':ncols * [C.dtype]}
            m = len(intersect1d(C.view(dtype), B.view(dtype)))
            if m >= 12:
                return a2b_offset
            assert(m>0)
    return None



def computeDistances(points):
    interdistances = []
    l = len(points)
    for i in range(l):
        for j in range(l):
            if  i!= j:
                a = points[i]
                b = points[j]
                c = a-b
                interdistances.append(c[0]*c[0]+c[1]*c[1]+c[2]*c[2])
    #print("%d interdistances computed" % (len(interdistances)))
    return interdistances
    


class Scanner:
    def __init__(s, name, orientation):
        s.name = name
        s.beacons =  None
        s.local2world = orientation
        s.world_offset = array([ 0, 0, 0], dtype=int32)
        s.interdistances = None
    
    def print(s):
        print(s.name)
        print(s.beacons)
        
    def getWorldBeacons(s):
        n,M = s.local2world
        r = s.beacons
        r = s.beacons.dot(M)
        r = r + s.world_offset        
        return r
        
# make list of scanners from text
def parse(text):
    scanners=[]
    for scanner in text.split("\n\n"):
        lines = scanner.split("\n")
        n = lines[0][4:].split(" ---")[0]
        s = Scanner(n, O[0])
        for p in lines[1:]:
            point = [int(x) for x in p.split(",")]
            point = array(point, dtype=int32)
            if s.beacons is None:
                s.beacons = point
            else:
                s.beacons = vstack( [s.beacons, point])
                
        s.interdistances = computeDistances(s.beacons)
        scanners.append(s)
    print("Read %d scanners"%(len(scanners)))
    
    return scanners

def parse_fn(fn):
    print("----parsing ", fn ,"----")
    return parse(open(fn).read())

def findcommon(A,B):
    r = 0
    for a in A:
        if a in B:
            r += 1
    return r

def complete_edges(n):
    return (n*(n-1))/2

def assemble2(scanners):
    unmapped = scanners
    mapped = []
    
    running = len(unmapped)**2
    while len(unmapped):
        if running <= 0:
            #No match
            return None                
        running -= 1
        
        #First scanner is used as origin
        if len(mapped) == 0:
            s=unmapped.pop(0)
            print(s.name, "is origin")
            s.local2world = O[0]
            s.world_offset = array([ 0, 0, 0], dtype=int32)    
            mapped.append(s)
        else:
            #cycle mapped scanners
            mapped_scanner = mapped.pop(0)
            mapped.append(mapped_scanner)
            
            #sort pool fitting mapped distances best
            for s in unmapped:
                s.distances_matched = findcommon(s.interdistances, mapped_scanner.interdistances)
            unmapped.sort(key=lambda x: x.distances_matched)            
            unmapped = list(reversed(unmapped))
            
            #take out scanner with distances fitting mapped scanner best
            unmapped_scanner = unmapped.pop(0)
            
            print("Best distance fit for %s is %s with %d fits" % (mapped_scanner.name.replace(" ",""), unmapped_scanner.name.replace(" ",""), unmapped_scanner.distances_matched))
            
            #flag to determine fate of unmapped scanner
            matched = False
            
            if unmapped_scanner.distances_matched >= complete_edges(12):
                #Try each of the 24 orientations
                for o in O:
                    #Try scanner with proposed offset and orientation
                    unmapped_scanner.local2world = o
                    unmapped_scanner.world_offset = array([0,0,0],dtype=int32)
                    offset2mapped = match(unmapped_scanner.getWorldBeacons(), mapped_scanner.getWorldBeacons())
                    if not offset2mapped is None:
                        print("%s's beacons mapped to %s points using %s and offset:" % (unmapped_scanner.name.replace(" ",""), mapped_scanner.name.replace(" ",""), n), offset2mapped, "scanners left:", len(unmapped))

                        #new found beacon resets search lease to rest of beacons
                        running = len(unmapped)*len(mapped)

                        #Apply offset to scanner to align with mapped points
                        unmapped_scanner.world_offset = offset2mapped
                        if 0:
                            #Verify that they align - remove for speed later
                            verify = match(unmapped_scanner.getWorldBeacons(), mapped_scanner.getWorldBeacons())
                            print(verify)
                            assert(not verify is None and verify[0] == 0 and verify[1] == 0 and verify[2] == 0 )

                        #print(mapped_points)
                        matched = True
                        #stop trying more orientations
                        break
            if matched:
                mapped.append(unmapped_scanner)
            else:
                #print("no match for ", unmapped_scanner.name)
                unmapped.append(unmapped_scanner)
        
    if len(unmapped) == 0:
        mapped_points = None
        for s in mapped:
            if mapped_points is None:
                mapped_points = s.getWorldBeacons()
            else:
                mapped_points = concatenate((mapped_points, s.getWorldBeacons()), axis=0)
                mapped_points = unique(mapped_points, axis=0)
        return mapped_points, mapped
                
    return None


def part1(beacons, c):
    num_rows, num_cols = beacons.shape
    v = num_rows
    print("There are %d beacons." % (v))
    if not c is None:
        assert(v == c)

def part2(scanners, c):
    l = len(scanners)
    largest = 0
    for i in range(l):
        for j in range(l):
            if i != j:
                
                v = sum(abs(scanners[i].world_offset - scanners[j].world_offset))
                if v > largest:
                    largest = v    
    print("Largest manhatten interdistance between scanners:", largest);
    if not c is None:
        assert(largest == c)



beacons, scanners = assemble2(parse_fn("i19_test.txt"))
part1(beacons, 79)
part2(scanners, 3621)

beacons, scanners = assemble2(parse_fn("i19.txt"))
part1(beacons, 454)
part2(scanners, 10813)

    

Spaces: +X+Y+Z +X+Z-Y +X-Y-Z +X-Z+Y -X+Y-Z -X-Z-Y -X-Y+Z -X+Z+Y +Z+Y-X +Z-X-Y +Z-Y+X +Z+X+Y -Z+Y+X -Z+X-Y -Z-Y-X -Z-X+Y +Y-X+Z +Y+Z+X +Y+X-Z +Y-Z-X -Y+X+Z -Y+Z-X -Y-X-Z -Y-Z+X
----parsing  i19_test.txt ----
Read 5 scanners
scanner 0 is origin
Best distance fit for scanner0 is scanner1 with 132 fits
scanner1's beacons mapped to scanner0 points using -Y-Z+X and offset: [   68 -1246   -43] scanners left: 3
Best distance fit for scanner0 is scanner4 with 30 fits
Best distance fit for scanner1 is scanner4 with 132 fits
scanner4's beacons mapped to scanner1 points using -Y-Z+X and offset: [  -20 -1133  1061] scanners left: 2
Best distance fit for scanner0 is scanner2 with 6 fits
Best distance fit for scanner1 is scanner3 with 132 fits
scanner3's beacons mapped to scanner1 points using -Y-Z+X and offset: [  -92 -2380   -20] scanners left: 1
Best distance fit for scanner4 is scanner2 with 132 fits
scanner2's beacons mapped to scanner4 points using -Y-Z+X and offset: [ 1105 -1205  1229] scanner

scanner22's beacons mapped to scanner34 points using -Y-Z+X and offset: [ 1045 -1158 -3701] scanners left: 13
Best distance fit for scanner5 is scanner25 with 0 fits
Best distance fit for scanner2 is scanner7 with 30 fits
Best distance fit for scanner12 is scanner7 with 132 fits
scanner7's beacons mapped to scanner12 points using -Y-Z+X and offset: [ 4650   -71 -3704] scanners left: 12
Best distance fit for scanner27 is scanner16 with 30 fits
Best distance fit for scanner29 is scanner3 with 30 fits
Best distance fit for scanner14 is scanner4 with 2 fits
Best distance fit for scanner0 is scanner4 with 0 fits
Best distance fit for scanner31 is scanner4 with 0 fits
Best distance fit for scanner20 is scanner4 with 0 fits
Best distance fit for scanner30 is scanner4 with 0 fits
Best distance fit for scanner13 is scanner15 with 2 fits
Best distance fit for scanner36 is scanner15 with 132 fits
scanner15's beacons mapped to scanner36 points using -Y-Z+X and offset: [ 5892    -6 -1222] scanners 